In [179]:
import pandas as pd
import numpy as np
from sklearn import svm

In [187]:
#Creates Lagged series
#Goes through a series and generates an lag+1 dimensional pandas DataFrame that has each previous lag timeunit
#as a column and current as the last cobilumn
#Input: Pandas Series
#Output: lag+1 dimensional DataFrame

def timeseriesLagged(data, lag=60):
    df = data
    columns = [df.shift(i) for i in range(1, lag+2)] 
    df = pd.concat(columns,axis=1)
    df.fillna(0, inplace=True)
    df.columns = [str(lag+2-x) for x in range(1,lag+2)]
    # df.reset_index(inplace=True,drop=False)
    df = df[df.columns[::-1]] #Flip because we want newer data on the right
    df= df.iloc[lag+1:] # drop the first 'lag' columns because zeroes.
    df.reset_index(drop=True,inplace=True)
    return df

# Binarizes the last column into 1, 0, -1. 1 = buy 0 = do nothing -1 = sell
# Rate is the percent increase or decrease that should trigger a buy or a sell
# lag is the time unit of lag. 
# atleast is how many of the lookahead need to be atleast the same or greater than flat+rat
# Input: lagged pandas DataFrame, uint lag, double dif, double flat, double atleast between 0 and 1
# Output : Pandas Dataframe with last column binarized
def binarizeTime(resLagged,rate = 0,lookahead = 0, flat = 0,atleast = 0.5):
    if lookahead <= 0 :
        raise Exception("lookahead Must be 1 or higher!")
    resLagged = resLagged.copy() # Make a deep copy
    last = np.shape(resLagged)[1] # find the length of the data 
    last = last-lookahead # convert it to string for loc
    colsLookAhead = list(resLagged.loc[:,str(last+1):str(last + lookahead)])
    colsLast = resLagged[str(last)]
    diffs = resLagged[colsLookAhead].subtract(colsLast,axis=0)
#     print(diffs)
    greater = diffs>=flat  # all the times the price changed higer than flat
    greater = np.count_nonzero(greater,axis=1).reshape((1,-1))
    lesser = diffs<=-flat # all the times the price fell lower than fat
    lesser = np.count_nonzero(lesser,axis=1).reshape((1,-1))
#     return greater,lesser
#     print(greater)
    greater = greater.reshape(1,-1)
    changeToBuy = np.any(greater > lesser & np.greater(greater,atleast*lookahead),axis=0) # make sure more rises than falls and atleast half rises
    changeToSell = np.any(lesser > greater & np.greater(lesser,atleast*lookahead),axis=0)      # make sure more falls than rises and atleast half rises
    changeToHold = ~changeToBuy & ~changeToSell
    resLagged = resLagged.drop(colsLookAhead,1)
    resLagged.loc[changeToSell,str(last+1)] = -1 # Set sell to -1
    resLagged.loc[changeToBuy,str(last+1)] = 1 # Set buy to 1
    resLagged.loc[changeToHold,str(last+1)] = 0 # Set to 0
    return resLagged

In [190]:
# Nifty
fut = pd.read_csv("Nifty50FUT.csv")
fut['Price']= fut['Price'].str.replace(",","").astype(np.double)
fut['Open']= fut['Open'].str.replace(",","").astype(np.double)
fut['High']= fut['High'].str.replace(",","").astype(np.double)
fut['Low']= fut['Low'].str.replace(",","").astype(np.double)
fut['Vol.']= fut['Vol.'].str.replace(",","").str.replace("M","e6").str.replace("-","0").str.replace("K","e3").astype(np.double)
fut = fut[::-1]
fut.reset_index(inplace=True, drop = True)

nifty = pd.read_csv("Nifty.csv")
nifty = nifty[::-1]
nifty.reset_index(inplace=True, drop = True)
nifty['Price']= nifty['Price'].str.replace(",","").astype(np.double)
nifty['Open']= nifty['Open'].str.replace(",","").astype(np.double)
nifty['High']= nifty['High'].str.replace(",","").astype(np.double)
nifty['Low']= nifty['Low'].str.replace(",","").astype(np.double)
nifty['Vol.']= nifty['Vol.'].str.replace(",","").str.replace("M","e6").str.replace("-","0").str.replace("K","e3").astype(np.double)


niftyDrop = np.setdiff1d(fut['Date'].values,nifty['Date'].values)
futDrop = np.setdiff1d(nifty['Date'].values,fut['Date'].values)
nifty = nifty[~nifty.Date.isin(futDrop)]
fut = fut[~fut.Date.isin(niftyDrop)]

In [193]:
volNifty = nifty['Vol.']
openNifty = nifty['Open']
highNifty = nifty['High']
lowNifty = nifty['Low']
dataNifty = nifty['Price']

volFut = fut['Vol.']
openFut = fut['Open']
highFut = fut['High']
lowFut = fut['Low']
dataFut = fut['Price']

In [192]:
lag=14
lookahead = 1
flat = 1
closeNifty = timeseriesLagged(dataNifty,lag + lookahead-1)
openNifty = timeseriesLagged(openNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
highNifty = timeseriesLagged(highNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
lowNifty = timeseriesLagged(lowNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
volNifty = timeseriesLagged(volNifty,lag + lookahead-1).drop(str(lag+1),axis=1)



,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,110350000.0,156360000.0,147630000.0,139610000.0,162560000.0,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0
1,156360000.0,147630000.0,139610000.0,162560000.0,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0
2,147630000.0,139610000.0,162560000.0,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0
3,139610000.0,162560000.0,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0
4,162560000.0,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0
5,120850000.0,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0,94470000.0
6,133660000.0,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0,94470000.0,98430000.0
7,125140000.0,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0,94470000.0,98430000.0,101260000.0
8,110910000.0,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0,94470000.0,98430000.0,101260000.0,187490000.0
9,117680000.0,152500000.0,127180000.0,135740000.0,211340000.0,200600000.0,172630000.0,145350000.0,105880000.0,94470000.0,98430000.0,101260000.0,187490000.0,106700000.0


In [169]:
futVals = np.array([
           fut.Open.values.astype(np.double), 
           fut.High.values.astype(np.double),
           fut.Low.values.astype(np.double),
           fut.Price.values.astype(np.double),
           fut['Vol.'].values.astype(np.double)
                   ])
niftyVals = np.array([
             nifty.Open.values.astype(np.double),
             nifty.High.values.astype(np.double),
             nifty.Low.values.astype(np.double),
             nifty.Price.values.astype(np.double),
             nifty['Vol.'].values.astype(np.double)
                ])

In [176]:
premiums = futVals[0:4]-niftyVals[0:4]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,6179.40,6178.55,6274.30,6279.10,6287.85,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30
1,6178.55,6274.30,6279.10,6287.85,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40
2,6274.30,6279.10,6287.85,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45
3,6279.10,6287.85,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35
4,6287.85,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10
5,6272.00,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10,5280.80
6,6156.95,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10,5280.80,5167.60
7,6200.10,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10,5280.80,5167.60,5137.45
8,6206.80,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10,5280.80,5167.60,5137.45,5317.25
9,6074.25,5935.75,5913.20,5705.30,5208.80,4899.30,5203.40,5033.45,5383.35,5274.10,5280.80,5167.60,5137.45,5317.25,5463.50


,Date,Price,Open,High,Low,Vol.,Change %
0,"Jan 02, 2008","6,179.40","6,144.70","6,197.00","6,060.85",110.35M,0.57%
1,"Jan 03, 2008","6,178.55","6,184.25","6,230.15","6,126.40",156.36M,-0.01%
2,"Jan 04, 2008","6,274.30","6,179.10","6,300.05","6,179.10",147.63M,1.55%
3,"Jan 07, 2008","6,279.10","6,271.00","6,289.80","6,193.35",139.61M,0.08%
4,"Jan 08, 2008","6,287.85","6,282.45","6,357.10","6,221.60",162.56M,0.14%
5,"Jan 09, 2008","6,272.00","6,287.55","6,338.30","6,231.25",120.85M,-0.25%
6,"Jan 10, 2008","6,156.95","6,278.10","6,347.00","6,142.90",133.66M,-1.83%
7,"Jan 11, 2008","6,200.10","6,166.65","6,224.20","6,112.55",125.14M,0.70%
8,"Jan 14, 2008","6,206.80","6,208.80","6,244.15","6,172.00",110.91M,0.11%
9,"Jan 15, 2008","6,074.25","6,226.35","6,260.45","6,053.30",117.68M,-2.14%
